In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm


def load_files(csv_file="",npy_file=""): return pd.read_csv(csv_file), np.load(npy_file, allow_pickle=True)

In [2]:
connections, embeddings = load_files(csv_file="connections.csv", npy_file="connections_embeddings.npy")
print(embeddings.shape, connections.shape)
connections.head()

labels_list = [1 if connection_true else 0 for connection_true in connections['true_attempt']]

(10000, 256) (10000, 5)


,logins,passwords,ips,dates,true_attempt
0,malik_antonia@email.com,Spenc&r4Ever,170.252.165.242,20:22:59 25-12-2020,False
1,josue_mariam@email.com,03031974,193.251.224.173,04:45:06 11-04-2022,False
2,laila_delilah@email.com,LDE1ILAH76,34.5.41.175,11:11:55 07-04-2023,False
3,charles_eliyahu@email.com,PATRICK1985,132.245.193.248,07:46:50 18-01-2022,False
4,sade_miguel@email.com,pRw1wPP,34.113.135.64,17:32:41 23-03-2023,False


In [ ]:

#accuracies = []
#neighbors = []
#for N in tqdm(range(1,30)):
    #clf = DecisionTreeClassifier(max_depth=N)
    #clf.fit(data_train, labels_train)
    #score = clf.score(data_test, labels_test)
    #neighbors.append(N)
    #accuracies.append(score)
    


#plt.plot(neighbors, accuracies)
#plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

labels_list = [1 if connection_true else 0 for connection_true in connections['true_attempt']]
labels = np.array(labels_list) #we transform the list in a np.array to ease the next steps (idk why, but sklearn seems to like it...)
#data_train, data_test, labels_train, labels_test = train_test_split(embeddings, labels, test_size=0.20, shuffle=True)

In [ ]:
import tensoflow as tf
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Dense, Input, Embedding, GlobalAveragePooling1D
from tensorflow.datasets import imdb
from tensorflow.keras.models import Sequential, Model
import numpy as np
import warnings

warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([Dense(ff_dim, activation='relu'), Dense(embed_dim)])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
    
    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)
    
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
vocab_size = 20000
maxlen = 200



In [ ]:
(x_train, y_train), (x_val, y_val) = imdb.load_data(num_words=vocab_size)
print(len(x_train), 'Training sequences')
print(len(x_val), 'Validation sequences')

In [ ]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = tf.keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

In [ ]:
# WHAT IF WE DONT USE THE EMBEDDING AND WE JUST USE NLP